# Libraries

In [45]:
import json
import pandas as pd
from openai import OpenAI

# Config

In [46]:
client = OpenAI(api_key=json.load(open('secrets.json'))['OPENAPI_SECRET_KEY'])
data_path = 'data/go_emotions_sun/neutral_single_50_comb_0.csv'

# Data

## Load Data

In [47]:
df = pd.read_csv(data_path)
print("DF size:", len(df))
df.head()

DF size: 50


,id,text,marah,jijik,takut,senang,sedih,terkejut,biasa,emotion
0,ee54dcb,"haha, rape",0,0,0,0,0,0,1,biasa
1,edkr19d,That's what they're for! I wouldn't use it for...,0,0,0,0,0,0,1,biasa
2,efcsmiw,"Damn, hoped i had missed some fan translations...",0,0,0,0,0,0,1,biasa
3,ee43fc3,Kid has profanity on the front of his shirt......,0,0,0,0,0,0,1,biasa
4,ef4fl50,reddit was having server issues,0,0,0,0,0,0,1,biasa


## Translate Data

In [48]:
def translate_text(text, emotion, target_lang="Sunda"):
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            # {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                # "content": f"Terjemahkan teks ini dari bahasa Inggris ke bahasa {target_lang}: {text}",
                'content': f""""Terjemahkan teks dari bahasa Inggris ke bahasa {target_lang} untuk dataset klasifikasi emosi. Cukup respons dengan teks terjemahan saja

emosi: {emotion}
teks: {text}
terjemahan teks:""",
            },
        ],
        max_tokens=100,
    )
    return completion.choices[0].message.content

for i, row in df.iterrows():
    print(f"# {i+1} of {len(df)}")
    print("emosi:", row['emotion'])
    print("teks:", row['text'])
    text_translated = translate_text(row['text'], row['emotion'], target_lang="Sunda")
    print("teks terjemahan:", text_translated)
    df.at[i, 'text_translated'] = text_translated
    print()

# 1 of 50
emosi: biasa
teks: haha, rape
teks terjemahan: haha, perkosa

# 2 of 50
emosi: biasa
teks: That's what they're for! I wouldn't use it for anything else. And make sure you don't have plans 6-10 hours from taking it...
teks terjemahan: Éta mah makéna keur kitu! Kuring moal dipaké keur naon waé anu sanés. Jeung pastikeun anjeun teu gaduh rencana 6-10 jam ti saprak nginum éta...

# 3 of 50
emosi: biasa
teks: Damn, hoped i had missed some fan translations or something...
teks terjemahan: Duh, miharep kuring sono kana sababaraha tarjamahan kipas atanapi naon waé...

# 4 of 50
emosi: biasa
teks: Kid has profanity on the front of his shirt..... Gatorfan
teks terjemahan: Budak téh aya kecap jorang dina hareupeun kaosna..... Gatorfan

# 5 of 50
emosi: biasa
teks: reddit was having server issues
teks terjemahan: reddit keur boga masalah server

# 6 of 50
emosi: biasa
teks: Most do, some are more reactable than others, like warlord and kensei, some are ridiculous like shinobi or conq or 

## Save Data

In [49]:
save_path = f'{data_path.split('.')[-0]}_translated.csv'
df.to_csv(save_path, index=False)
print("Saved to:", save_path)

Saved to: data/go_emotions_sun/neutral_single_50_comb_0_translated.csv
